In [5]:
import pandas as pd
import openai
import os
from dotenv import load_dotenv

# Charger la clé API depuis le fichier .env
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Créer le dossier output s'il n'existe pas
output_dir = "../output"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Fonction pour envoyer une requête avec un pre-prompt
def ask_gpt_with_preprompt(pre_prompt, user_query, model="gpt-4-0613", max_tokens=500):
    try:
        # Messages comprenant le pre-prompt et la question utilisateur
        messages = [
            {"role": "system", "content": pre_prompt},  # Pre-prompt
            {"role": "user", "content": user_query}    # Question utilisateur
        ]
        
        # Requête à l'API
        response = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            max_tokens=max_tokens,
            temperature=0.7  # Ajuste la créativité du modèle
        )
        
        # Retourne la réponse du modèle
        return response.choices[0].message.content.strip()
    
    except Exception as e:
        return f"Erreur lors de l'appel à l'API : {e}"

# Pre-prompt pour guider GPT-4
pre_prompt = (
    "Tu es un analyste financier pour le secteur bancaire. "
    "Analyse les données hebdomadaires d'agences d'une banque et rédige un compte rendu clair et synthétique. "
    "Différencie les clients particuliers (individus avec comptes personnels) et professionnels (PME, SARL, SASU). "
    "Base-toi sur les champs fournis pour ton analyse."
)

# Fonction pour traiter une agence, générer un compte rendu et l'écrire dans un fichier
def process_agency(row):
    bank_name = row["LIB_STR_NIV2"]
    num_semaine = row["ID_SEM_COMM"]
    occ_part = row["OCC_PART"]
    conso_dec = row["CONSO_DEC"]
    bilan_net = row["BILAN_NET"]
    vers_ep_fi = row["FI_NET"]
    vb_iard_part = row["VB_IARD_PART"]
    vb_prev_part = row["VB_PREV_PART"]
    vb_ass_vie_peri_pea_hisseo = row["VB_ASS_PERI_PEA_HISS"]
    moy_part = row["MOY_TX_PART"]
    somme_part = row["S_RG_PART"]
    rang_part = row["RANG_PART"]
    occ_pro = row["OCC_PRO"]
    occ_pro_cible = row["OCC_PRO_CIBLE"]
    eqpt_dec = row["EQPT_DEC"]
    vb_iard_pro = row["VB_IARD_PRO"]
    vb_prev_pro = row["VB_PREV_PRO"]
    vb_contr_comm = row["VB_CONTR_COMM"]
    moy_pro = row["MOY_TX_PRO"]
    somme_pro = row["S_RG_PRO"]
    rang_pro = row["RANG_PRO"]
    moy = row["MOY"]
    somme_rang = row["S_RG"]
    rang = row["RANG"]

    # Construire la question utilisateur à partir des données de la banque
    user_query = (
        f"Données de la banque {bank_name}, semaine {num_semaine} :\n"
        f"- Ouvertures comptes particuliers : {occ_part}\n"
        f"- Prêts consommation décaissés : {conso_dec} €\n"
        f"- Collecte bilantielle nette : {bilan_net} €\n"
        f"- Versements épargne financière : {vers_ep_fi} €\n"
        f"- Ventes brutes assurances IARD particuliers : {vb_iard_part}\n"
        f"- Ventes brutes prévoyance particuliers : {vb_prev_part}\n"
        f"- Ventes brutes assurances vie et produits associés : {vb_ass_vie_peri_pea_hisseo}\n"
        f"- Moyenne taux atteinte objectifs particuliers : {moy_part} %\n"
        f"- Somme des rangs particuliers : {somme_part}\n"
        f"- Rang dans segment particuliers : {rang_part}\n"
        f"- Ouvertures comptes professionnels : {occ_pro}\n"
        f"- Ouvertures comptes professionnels ciblés : {occ_pro_cible}\n"
        f"- Crédits équipement décaissés : {eqpt_dec} €\n"
        f"- Ventes brutes assurances IARD professionnels : {vb_iard_pro}\n"
        f"- Ventes brutes prévoyance professionnels : {vb_prev_pro}\n"
        f"- Ventes brutes contrats commerçants : {vb_contr_comm}\n"
        f"- Moyenne taux atteinte objectifs professionnels : {moy_pro} %\n"
        f"- Somme des rangs professionnels : {somme_pro}\n"
        f"- Rang dans segment professionnels : {rang_pro}\n"
        f"- Moyenne taux atteinte global (particuliers et professionnels) : {moy} %\n"
        f"- Somme des rangs globaux : {somme_rang}\n"
        f"- Rang final agence (13 items cumulés) : {rang}\n\n"
        "Rédige un compte rendu synthétique et pertinent pour le directeur de cette agence."
    )

    # Appeler GPT pour générer le compte rendu
    report = ask_gpt_with_preprompt(pre_prompt, user_query)

    # Enregistrer le rapport dans un fichier
    filename = f"{output_dir}/mail_{bank_name.replace(' ', '_')}.txt"
    try:
        with open(filename, "w", encoding="utf-8") as file:
            file.write(report)
        print(f"Rapport pour {bank_name} sauvegardé dans {filename}")
    except Exception as e:
        print(f"Erreur lors de la sauvegarde du rapport pour {bank_name}: {e}")

# Lire les données depuis le fichier Excel
file_path = "../sources/donnéesSorties.xlsx"
df = pd.read_excel(file_path)

# Traiter les agences une par une
for _, row in df.iterrows():
    process_agency(row)


Rapport pour BANQUE A sauvegardé dans ../output/mail_BANQUE_A.txt
Rapport pour BANQUE B sauvegardé dans ../output/mail_BANQUE_B.txt
Rapport pour BANQUE C sauvegardé dans ../output/mail_BANQUE_C.txt


KeyboardInterrupt: 